In [ ]:
pip install requests pandas

In [ ]:
pip install requests beautifulsoup4 pandas

In [ ]:
import requests
import pandas as pd
import time

# ⚠️ [필수] 발급받은 RAWG API 키를 여기에 입력하세요.
# 이 키는 외부에 노출되지 않도록 주의해야 합니다.
RAWG_API_KEY = "d3faf044b8644c259fe6164d74e5f610"

# 1. API 기본 설정
BASE_URL = "https://api.rawg.io/api/games"

# 2. 데이터 수집 함수 정의
def fetch_rawg_data(page_size=40, max_pages=125):
    """
    RAWG API에서 게임 데이터를 페이지별로 수집하여 리스트에 저장합니다.

    :param page_size: 페이지당 가져올 게임 수 (최대 40)
    :param max_pages: 가져올 최대 페이지 수
    :return: 수집된 모든 게임 데이터(딕셔너리 형태) 리스트
    """

    ordering_criteria = "-added"
    all_games_data = []

    for page in range(1, max_pages + 1):
        params = {
            "key": RAWG_API_KEY,
            "page": page,
            "page_size": page_size,
            "ordering": ordering_criteria
        }

        try:
            print(f"--- 페이지 {page} 데이터 요청 중... ---")
            response = requests.get(BASE_URL, params=params)
            response.raise_for_status() # HTTP 오류가 발생하면 예외 발생

            data = response.json()

            # 수집된 게임 목록을 리스트에 추가
            all_games_data.extend(data.get("results", []))

            # 다음 페이지 URL이 없으면 루프 종료
            if not data.get("next"):
                print("마지막 페이지입니다. 수집 종료.")
                break

            # API 호출 제한을 피하기 위해 잠시 대기
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"API 요청 오류 발생: {e}")
            break

    return all_games_data

# 3. 데이터 추출 및 DataFrame 생성 함수
def process_data(data_list):
    """
    수집된 원본 데이터에서 필요한 항목만 추출하여 DataFrame을 생성합니다.
    """
    processed_list = []
    rating_categories = ["exceptional", "recommended", "meh", "skip"]

    for game in data_list:
        # '장르'와 '플랫폼'은 리스트 형태로 오므로 쉼표로 연결하여 문자열로 변환
        genres = ", ".join([g['name'] for g in game.get('genres', [])])
        platforms = ", ".join([p['platform']['name'] for p in game.get('platforms', []) if p.get('platform')])

        processed_list.append({
            'game_id': game.get('id'),
            'title': game.get('name'),
            'released_date': game.get('released'),
            'rating': game.get('rating'),
            'rating_count': game.get('ratings_count'),
            'genres': genres,
            'platforms': platforms,
            'slug': game.get('slug') # 상세 정보 크롤링을 위한 고유 주소명
        })

        category_counts = {f'vote_{cat}': 0 for cat in rating_categories}

        for rating_info in game.get('ratings', []):
            title = rating_info.get('title')
            count = rating_info.get('count')

            if title in rating_categories:
                # 'vote_exceptional', 'vote_recommended' 등의 컬럼 이름으로 저장
                category_counts[f'vote_{title}'] = count

        # 3. 기본 정보와 투표 수 결합
        processed_list.update(category_counts)

    return pd.DataFrame(processed_list)

# 4. 메인 실행 및 저장
if __name__ == "__main__":

    # ⚠️ 반드시 API 키를 YOUR_API_KEY_HERE 대신 입력해야 합니다.
    if RAWG_API_KEY == "d3faf044b8644c259fe6164d74e5f610":
        print("에러: RAWG_API_KEY를 유효한 키로 변경해주세요.")
    else:
        # 10페이지 (400개 게임) 수집 시도
        raw_data = fetch_rawg_data(page_size=40, max_pages=125)

        if raw_data:
            df = process_data(raw_data)

            # 5. CSV 파일로 저장
            filename = f"rawg_games_data_{len(df)}.csv"
            df.to_csv(filename, index=False, encoding='utf-8-sig')

            print("\n==============================================")
            print(f"데이터 수집 및 저장이 완료되었습니다. 파일명: {filename}")
            print(f"총 수집된 게임 수: {len(df)}")
            print("==============================================")
            print("\n--- 저장된 데이터프레임 미리보기 ---")
            print(df.head())

에러: RAWG_API_KEY를 유효한 키로 변경해주세요.


In [ ]:
import requests
import pandas as pd
import time
import numpy as np

# ⚠️ [필수] 발급받은 RAWG API 키를 여기에 입력하세요.
RAWG_API_KEY = "b008c6fe98de43f6ad6a4b93ad5ba439"

def fetch_game_storyline(game_id):
    """
    RAWG API의 상세 호출을 통해 특정 게임의 줄거리를 가져오는 함수
    """
    url = f"https://api.rawg.io/api/games/{game_id}"
    params = {"key": RAWG_API_KEY}

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        # description_raw: HTML 태그가 제거된 순수 텍스트 줄거리
        storyline = data.get('description_raw', np.nan)

        # 줄거리 텍스트가 너무 짧거나(50자 미만) NaN일 경우 다시 NaN 반환
        if isinstance(storyline, str) and len(storyline) < 50:
             return np.nan

        return storyline

    except requests.exceptions.RequestException as e:
        # 오류 발생 시 해당 셀에 NaN을 반환하여 나중에 재처리할 수 있도록 합니다.
        print(f"[{game_id}] API 요청 오류 발생: {e}")
        return np.nan
    except Exception as e:
        print(f"[{game_id}] 일반 오류 발생: {e}")
        return np.nan
try:
    # 이전에 저장한 CSV 파일이 있다면 로드
    df = pd.read_csv('rawg_games_data_5000.csv')
except FileNotFoundError:
    # 파일이 없다면 테스트용 DataFrame 생성 (실제 사용 시 API 호출로 df를 생성해야 합니다)
    print("경고: 'rawg_games_data_5000.csv' 파일을 찾을 수 없습니다. 테스트용 DataFrame을 생성합니다.")
    df = pd.DataFrame({
        'game_id': [3498, 3328, 41494, 4200], # GTA V, Witcher 3, Cyberpunk, Portal 2
        'title': ['Grand Theft Auto V', 'The Witcher 3: Wild Hunt', 'Cyberpunk 2077', 'Portal 2'],
        'released_date': ['2013-09-17', '2015-05-18', '2020-12-10', '2011-04-18']
    })

In [ ]:
from google.colab import files
files.download("/content/rawg_games_data_5000.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def collect_storylines(df, sleep_sec=0.5):
    """
    DataFrame의 모든 게임에 대해 줄거리 정보를 수집하고 새로운 컬럼을 추가합니다.

    :param df: game_id 컬럼을 포함한 DataFrame
    :param sleep_sec: API 호출 사이의 대기 시간 (초)
    :return: 'storyline' 컬럼이 추가된 DataFrame
    """

    # 🌟 'storyline' 컬럼을 초기화합니다.
    df['storyline'] = np.nan

    for index, row in df.iterrows():
        game_id = row['game_id']
        title = row['title']

        # 이미 줄거리가 있는 경우 (이전에 부분적으로 수집했을 경우) 건너뜁니다.
        if pd.notna(row.get('storyline')):
             print(f"[{index+1}/{len(df)}] {title} (ID: {game_id}) - 이미 줄거리 있음. 건너뜀.")
             continue

        print(f"[{index+1}/{len(df)}] {title} (ID: {game_id}) - 상세 정보 요청 중...")

        # API 호출
        plot_text = fetch_game_storyline(game_id)

        # 줄거리 결과를 DataFrame에 저장
        df.loc[index, 'storyline'] = plot_text

        # ⚠️ API 호출 제한을 피하기 위한 필수 대기
        time.sleep(sleep_sec)

    return df

# 3. 메인 실행 및 저장
if __name__ == "__main__":
    if RAWG_API_KEY == "3a8261cf9ca447b4a2e5f21d276b7f5d":
        print("에러: RAWG_API_KEY를 유효한 키로 변경해주세요.")
    else:
        # 줄거리 수집 시작 (API 부하를 줄이기 위해 0.5초 대기 설정)
        df_with_storylines = collect_storylines(df, sleep_sec=0.5)

        # 4. 최종 결과 확인 및 저장
        output_filename = "rawg_games_data_with_storyline.csv"
        df_with_storylines.to_csv(output_filename, index=False, encoding='utf-8-sig')

        print("\n==============================================")
        print("✅ 줄거리 수집 및 저장이 완료되었습니다.")
        print(f"파일 저장 위치: {output_filename}")
        print("==============================================")
        print(df_with_storylines[['title', 'released_date', 'storyline']].head())

[1/5000] Grand Theft Auto V (ID: 3498) - 상세 정보 요청 중...


/tmp/ipython-input-3234804547.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Rockstar Games went bigger, since their previous installment of the series. You get the complicated and realistic world-building from Liberty City of GTA4 in the setting of lively and diverse Los Santos, from an old fan favorite GTA San Andreas. 561 different vehicles (including every transport you can operate) and the amount is rising with every update. 
Simultaneous storytelling from three unique perspectives: 
Follow Michael, ex-criminal living his life of leisure away from the past, Franklin, a kid that seeks the better future, and Trevor, the exact past Michael is trying to run away from. 
GTA Online will provide a lot of additional challenge even for the experienced players, coming fresh from the story mode. Now you will have other players around that can help you just as likely as ruin your mission. Every GTA mechan

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[19/5000] Limbo (ID: 1030) - 상세 정보 요청 중...
[20/5000] Team Fortress 2 (ID: 11859) - 상세 정보 요청 중...
[21/5000] DOOM (2016) (ID: 2454) - 상세 정보 요청 중...
[22/5000] Cyberpunk 2077 (ID: 41494) - 상세 정보 요청 중...
[23/5000] Terraria (ID: 422) - 상세 정보 요청 중...
[24/5000] Dota 2 (ID: 10213) - 상세 정보 요청 중...
[25/5000] Warframe (ID: 766) - 상세 정보 요청 중...
[26/5000] Grand Theft Auto IV (ID: 4459) - 상세 정보 요청 중...
[27/5000] Rocket League (ID: 3272) - 상세 정보 요청 중...
[28/5000] Horizon Zero Dawn (ID: 278) - 상세 정보 요청 중...
[29/5000] Metro 2033 (ID: 29028) - 상세 정보 요청 중...
[30/5000] Rise of the Tomb Raider (ID: 7689) - 상세 정보 요청 중...
[31/5000] Batman: Arkham Knight (ID: 3287) - 상세 정보 요청 중...
[32/5000] Metal Gear Solid V: The Phantom Pain (ID: 3192) - 상세 정보 요청 중...
[33/5000] The Witcher 2: Assassins of Kings Enhanced Edition (ID: 16944) - 상세 정보 요청 중...
[34/5000] Apex Legends (ID: 290856) - 상세 정보 요청 중...
[35/5000] Grand Theft Auto: San Andreas (ID: 416) - 상세 정보 요청 중...
[36/5000] The Witc

In [ ]:
from google.colab import files
files.download("/content/rawg_games_data_with_storyline.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

리뷰 수집

In [ ]:
import pandas as pd
import requests
import time
import numpy as np
import os

# ⚠️ RAWG_API_KEY가 반드시 설정되어 있어야 합니다.
RAWG_API_KEY = "d3faf044b8644c259fe6164d74e5f610"

# ----------------------------------------------------
# 1. 투표 데이터 수집 및 처리 함수
# ----------------------------------------------------
def fetch_vote_data(game_id):
    """ RAWG API 상세 호출을 통해 투표 정보를 가져오는 함수 """
    url = f"https://api.rawg.io/api/games/{game_id}"
    params = {"key": RAWG_API_KEY}

    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        rating_categories = ["exceptional", "recommended", "meh", "skip"]
        game_votes = {f'vote_{cat}': 0 for cat in rating_categories}

        for rating_info in data.get('ratings', []):
            title = rating_info.get('title')
            count = rating_info.get('count')
            if title in rating_categories:
                game_votes[f'vote_{title}'] = count

        return game_votes

    except requests.exceptions.HTTPError as e:
        # 401/403 오류 (키 문제) 출력
        if e.response.status_code in [401, 403]:
            print(f"\nFATAL ERROR: API Key 문제 발생. 상태 코드: {e.response.status_code}")
        return None
    except Exception:
        return None

def fetch_and_process_votes(df, sleep_sec=0.5):
    """ DataFrame 전체를 순회하며 투표 정보를 수집하고 업데이트합니다. """
    vote_cols = [f'vote_{cat}' for cat in ["exceptional", "recommended", "meh", "skip"]]

    # 🌟 기존 컬럼이 없으면 NaN으로 초기화
    for col in vote_cols:
        if col not in df.columns:
            df[col] = np.nan

    updated_rows = []

    # 🌟🌟🌟 전체 DataFrame을 대상으로 처리합니다. 🌟🌟🌟
    df_to_process = df.copy()

    print(f"--- 총 {len(df_to_process)}개 게임의 상세 투표 정보 수집 시작 (예상 시간: 약 41분) ---")

    for index, row in df_to_process.iterrows():
        game_id = row['game_id']

        # ⚠️ 이미 유효한 투표 정보가 있다면 API 호출을 건너뛰어 효율을 높입니다.
        # vote_exceptional 컬럼이 NaN이 아닐 경우 건너뜁니다.
        if pd.notna(row.get('vote_exceptional')):
             updated_rows.append(row.copy())
             continue

        # API 호출
        vote_data = fetch_vote_data(game_id)

        updated_row = row.copy()
        if vote_data:
            updated_row.update(vote_data)

        updated_rows.append(updated_row)

        print(f"[{index+1}/{len(df_to_process)}] ID:{game_id} - 처리 완료. 다음 요청까지 대기 중...")
        time.sleep(sleep_sec)

    # 2. 결과 병합
    if updated_rows:
        # 인덱스를 사용하여 원본 df를 새로운 데이터로 대체합니다.
        df_updated = pd.DataFrame(updated_rows)
        # 🌟 인덱스를 맞추기 위해 원본 df의 인덱스를 새로 부여
        df_updated.index = df.index

        return df_updated
    return df # 업데이트된 데이터가 없으면 원본 반환

# ----------------------------------------------------
# 3. 메인 실행 및 저장
# ----------------------------------------------------
if __name__ == "__main__":

    # ⚠️ df 로드 (이전 단계에서 로드된 'rawg_games_data_with_storyline.csv'를 사용)
    try:
        df = pd.read_csv('rawg_games_data_with_storyline.csv')
    except FileNotFoundError:
        print("에러: 'rawg_games_data_with_storyline.csv' 파일을 찾을 수 없습니다.")
        exit()

    # 🌟 전체 5,000개 게임 수집 시작
    df_final = fetch_and_process_votes(df, sleep_sec=0.5)

    # 4. 결과 확인 및 저장
    output_filename = "rawg_games_data_FULL_with_votes.csv"
    df_final.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print("\n==============================================")
    print("✨✨✨ 전체 데이터 수집 및 보강 완료! ✨✨✨")
    print(f"총 {len(df_final)}개 게임 데이터가 저장되었습니다.")
    print(f"파일 위치: {output_filename}")
    print("==============================================")

    # 최종 데이터 요약 확인
    vote_cols = [col for col in df_final.columns if col.startswith('vote_')]
    if vote_cols:
        df_final[vote_cols] = df_final[vote_cols].apply(pd.to_numeric, errors='coerce').fillna(0)

        print("\n--- 전체 게임 투표 분포 요약 ---")
        print(df_final[vote_cols].sum().to_string())
        print("\n--- 새로 추가된 투표 컬럼 미리보기 (상위 5개) ---")
        print(df_final[['title', 'rating_count', 'vote_exceptional', 'vote_recommended']].head().to_string())

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[21/5000] ID:2454 - 처리 완료. 다음 요청까지 대기 중...
[22/5000] ID:41494 - 처리 완료. 다음 요청까지 대기 중...
[23/5000] ID:422 - 처리 완료. 다음 요청까지 대기 중...
[24/5000] ID:10213 - 처리 완료. 다음 요청까지 대기 중...
[25/5000] ID:766 - 처리 완료. 다음 요청까지 대기 중...
[26/5000] ID:4459 - 처리 완료. 다음 요청까지 대기 중...
[27/5000] ID:3272 - 처리 완료. 다음 요청까지 대기 중...
[28/5000] ID:278 - 처리 완료. 다음 요청까지 대기 중...
[29/5000] ID:29028 - 처리 완료. 다음 요청까지 대기 중...
[30/5000] ID:7689 - 처리 완료. 다음 요청까지 대기 중...
[31/5000] ID:3287 - 처리 완료. 다음 요청까지 대기 중...
[32/5000] ID:3192 - 처리 완료. 다음 요청까지 대기 중...
[33/5000] ID:16944 - 처리 완료. 다음 요청까지 대기 중...
[34/5000] ID:290856 - 처리 완료. 다음 요청까지 대기 중...
[35/5000] ID:416 - 처리 완료. 다음 요청까지 대기 중...
[36/5000] ID:17822 - 처리 완료. 다음 요청까지 대기 중...
[37/5000] ID:11973 - 처리 완료. 다음 요청까지 대기 중...
[38/5000] ID:9767 - 처리 완료. 다음 요청까지 대기 중...
[39/5000] ID:19103 - 처리 완료. 다음 요청까지 대기 중...
[40/5000] ID:23027 - 처리 완료. 다음 요청까지 대기 중...
[41/5000] ID:41 - 처리 완료. 다음 요청까지 대기 중...
[42/5000] ID:4427 - 처리 완료. 다음 요청까지 대기 중...
[43/5000] ID:1

In [ ]:
from google.colab import files
files.download("/content/rawg_games_data_FULL_with_votes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>